In [34]:
# Install required packages
!pip install pandas scikit-learn nltk tensorflow transformers geopy

import pandas as pd
import numpy as np
import re
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow import keras
from transformers import pipeline
import random

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [35]:
# Create synthetic dataset for EV charging station queries
data = {
    'text': [
        "find charging stations near me",
        "where can I charge my electric car",
        "nearest EV charging point",
        "show me charging stations in downtown",
        "EV chargers nearby",
        "find fast charging stations",
        "where is the closest charging station",
        "public charging points near my location",
        "find DC fast chargers",
        "level 2 charging stations nearby",
        "are there any charging stations open now",
        "24/7 EV charging near me",
        "charging stations with available slots",
        "find free charging stations",
        "paid EV charging points",
        "Tesla superchargers near me",
        "CCS compatible charging stations",
        "CHAdeMO charging points",
        "how far is the nearest charging station",
        "directions to EV charger"
    ],
    'intent': [
        'find_stations', 'find_stations', 'find_stations', 'find_stations', 'find_stations',
        'find_fast_charging', 'find_stations', 'find_stations', 'find_fast_charging', 'find_stations',
        'check_availability', 'check_availability', 'check_availability', 'filter_free', 'filter_paid',
        'filter_tesla', 'filter_ccs', 'filter_chademo', 'get_distance', 'get_directions'
    ]
}

df = pd.DataFrame(data)

# Add more training data
additional_data = {
    'text': [
        "I need to charge my EV",
        "where to plug in my electric vehicle",
        "fast charging options",
        "supercharger locations",
        "find me a charger",
        "available charging spots",
        "working charging stations",
        "EV charging near shopping mall",
        "charging stations with restaurants",
        "how to get to charging point"
    ],
    'intent': [
        'find_stations', 'find_stations', 'find_fast_charging', 'filter_tesla', 'find_stations',
        'check_availability', 'check_availability', 'find_stations', 'filter_amenities', 'get_directions'
    ]
}

df = pd.concat([df, pd.DataFrame(additional_data)], ignore_index=True)

In [36]:
class TextPreprocessor:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))

    def clean_text(self, text):
        # Convert to lowercase
        text = text.lower()

        # Remove special characters and digits
        text = re.sub(r'[^a-zA-Z\s]', '', text)

        # Tokenize
        tokens = word_tokenize(text)

        # Remove stopwords
        tokens = [token for token in tokens if token not in self.stop_words]

        return ' '.join(tokens)

# Initialize preprocessor
preprocessor = TextPreprocessor()

# Preprocess the text data
df['cleaned_text'] = df['text'].apply(preprocessor.clean_text)

print("Sample preprocessed data:")
print(df.head())

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
X = vectorizer.fit_transform(df['cleaned_text'])

# Encode labels
intent_mapping = {intent: idx for idx, intent in enumerate(df['intent'].unique())}
reverse_intent_mapping = {idx: intent for intent, idx in intent_mapping.items()}

y = df['intent'].map(intent_mapping)

print("Feature matrix shape:", X.shape)
print("Intent mapping:", intent_mapping)

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

# Predict and evaluate
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Naive Bayes Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=[reverse_intent_mapping[i] for i in sorted(reverse_intent_mapping.keys())]))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Convert sparse matrix to array for neural network
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Build neural network
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_dense.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(intent_mapping), activation='softmax')
])

nn_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = nn_model.fit(
    X_train_dense, y_train,
    epochs=50,
    batch_size=8,
    validation_data=(X_test_dense, y_test),
    verbose=1
)

# Evaluate neural network
nn_loss, nn_accuracy = nn_model.evaluate(X_test_dense, y_test)
print(f"Neural Network Accuracy: {nn_accuracy:.2f}")

In [ ]:
class EVChargingChatbot:
    def __init__(self, vectorizer, model, intent_mapping, reverse_intent_mapping):
        self.vectorizer = vectorizer
        self.model = model
        self.intent_mapping = intent_mapping
        self.reverse_intent_mapping = reverse_intent_mapping
        self.preprocessor = TextPreprocessor()

        # Mock charging station database
        self.charging_stations = [
            {"name": "EVGo Downtown", "type": "DC Fast", "available": True, "distance": "0.5 miles"},
            {"name": "ChargePoint Mall", "type": "Level 2", "available": True, "distance": "1.2 miles"},
            {"name": "Tesla Supercharger", "type": "Supercharger", "available": False, "distance": "2.1 miles"},
            {"name": "Electrify America", "type": "DC Fast", "available": True, "distance": "3.0 miles"}
        ]

    def predict_intent(self, text):
        # Preprocess text
        cleaned_text = self.preprocessor.clean_text(text)

        # Transform using TF-IDF
        text_vector = self.vectorizer.transform([cleaned_text])

        # Predict intent
        if hasattr(self.model, 'predict_proba'):
            # For traditional ML models
            prediction = self.model.predict(text_vector)[0]
            confidence = np.max(self.model.predict_proba(text_vector))
        else:
            # For neural network models
            prediction_proba = self.model.predict(text_vector.toarray())
            prediction = np.argmax(prediction_proba, axis=1)[0]
            confidence = np.max(prediction_proba)

        intent = self.reverse_intent_mapping[prediction]
        return intent, confidence

    def get_response(self, intent, user_input):
        intent_responses = {
            'find_stations': self._handle_find_stations,
            'find_fast_charging': self._handle_fast_charging,
            'check_availability': self._handle_check_availability,
            'filter_free': self._handle_filter_free,
            'filter_paid': self._handle_filter_paid,
            'filter_tesla': self._handle_filter_tesla,
            'filter_ccs': self._handle_filter_ccs,
            'filter_chademo': self._handle_filter_chademo,
            'get_distance': self._handle_get_distance,
            'get_directions': self._handle_get_directions,
            'filter_amenities': self._handle_filter_amenities
        }

        handler = intent_responses.get(intent, self._handle_unknown)
        return handler(user_input)

    def _handle_find_stations(self, user_input):
        stations = [s for s in self.charging_stations if s['available']]
        response = "I found these charging stations near you:\n"
        for station in stations[:3]:
            response += f"• {station['name']} ({station['type']}) - {station['distance']} away\n"
        return response

    def _handle_fast_charging(self, user_input):
        fast_stations = [s for s in self.charging_stations if 'Fast' in s['type'] or 'Supercharger' in s['type']]
        response = "Fast charging stations available:\n"
        for station in fast_stations:
            status = "Available" if station['available'] else "Occupied"
            response += f"• {station['name']} - {station['distance']} - {status}\n"
        return response

    def _handle_check_availability(self, user_input):
        available_stations = [s for s in self.charging_stations if s['available']]
        response = f"Found {len(available_stations)} available charging stations:\n"
        for station in available_stations:
            response += f"• {station['name']} - {station['distance']}\n"
        return response

    def _handle_filter_tesla(self, user_input):
        tesla_stations = [s for s in self.charging_stations if 'Tesla' in s['name']]
        response = "Tesla Superchargers:\n"
        for station in tesla_stations:
            response += f"• {station['name']} - {station['distance']}\n"
        return response

    def _handle_get_distance(self, user_input):
        nearest = min(self.charging_stations, key=lambda x: float(x['distance'].split()[0]))
        return f"The nearest charging station is {nearest['name']}, {nearest['distance']} away."

    def _handle_get_directions(self, user_input):
        return "I can provide directions to the nearest charging station. Please enable location services for precise navigation."

    def _handle_filter_free(self, user_input):
        return "Free charging stations are marked in the app. Currently, most public stations require payment."

    def _handle_filter_paid(self, user_input):
        return "Most public charging stations require payment. You can check pricing in the charging network apps."

    def _handle_filter_ccs(self, user_input):
        return "CCS compatible stations: EVGo Downtown and Electrify America support CCS connectors."

    def _handle_filter_chademo(self, user_input):
        return "CHAdeMO stations: EVGo Downtown supports CHAdeMO connectors."

    def _handle_filter_amenities(self, user_input):
        return "Charging stations with amenities: Mall locations typically have restaurants and shopping nearby."

    def _handle_unknown(self, user_input):
        return "I'm not sure how to help with that. I can help you find charging stations, check availability, or filter by type."

    def chat(self):
        print("EV Charging Station Finder Chatbot")
        print("Type 'quit' to exit\n")

        while True:
            user_input = input("You: ")
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("Chatbot: Goodbye! Safe travels!")
                break

            intent, confidence = self.predict_intent(user_input)
            print(f"Detected intent: {intent} (confidence: {confidence:.2f})")

            response = self.get_response(intent, user_input)
            print(f"Chatbot: {response}\n")

In [ ]:
# Initialize chatbot with the neural network model
chatbot = EVChargingChatbot(vectorizer, nn_model, intent_mapping, reverse_intent_mapping)

# Test with sample queries
test_queries = [
    "find charging stations",
    "where are fast chargers",
    "is there a Tesla supercharger nearby",
    "how far is the nearest station",
    "get me directions to charging point"
]

print("Testing Chatbot:\n")
for query in test_queries:
    print(f"User: {query}")
    intent, confidence = chatbot.predict_intent(query)
    response = chatbot.get_response(intent, query)
    print(f"Bot: {response}")
    print(f"Intent: {intent}, Confidence: {confidence:.2f}\n")

In [ ]:
# Uncomment to run interactive chat session
# chatbot.chat()

In [ ]:
# Save the trained models and vectorizer
import joblib
import pickle

# Save neural network model
nn_model.save('ev_chatbot_model.h5')

# Save vectorizer and mappings
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

with open('intent_mappings.pkl', 'wb') as f:
    pickle.dump({'intent_mapping': intent_mapping, 'reverse_mapping': reverse_intent_mapping}, f)

print("Models and artifacts saved successfully!")